# Mini Project 5-5 Explore Hypothesis Testing

## Introduction

You work for an environmental think tank called Repair Our Air (ROA). ROA is formulating policy recommendations to improve the air quality in America, using the Environmental Protection Agency's Air Quality Index (AQI) to guide their decision making. An AQI value close to 0 signals "little to no" public health concern, while higher values are associated with increased risk to public health. 

They've tasked you with leveraging AQI data to help them prioritize their strategy for improving air quality in America.

ROA is considering the following decisions. For each, construct a hypothesis test and an accompanying visualization, using your results of that test to make a recommendation:

1. ROA is considering a metropolitan-focused approach. Within California, they want to know if the mean AQI in Los Angeles County is statistically different from the rest of California.
2. With limited resources, ROA has to choose between New York and Ohio for their next regional office. Does New York have a lower AQI than Ohio?
3. A new policy will affect those states with a mean AQI of 10 or greater. Will Michigan be affected by this new policy?

**Notes:**
1. For your analysis, you'll default to a 5% level of significance.
2. Throughout the lab, for two-sample t-tests, use Welch's t-test (i.e., setting the `equal_var` parameter to `False` in `scipy.stats.ttest_ind()`). This will account for the possibly unequal variances between the two groups in the comparison.

## Step 1: Imports

To proceed with your analysis, import `pandas` and `numpy`. To conduct your hypothesis testing, import `stats` from `scipy`.

#### Import Packages

In [1]:
# Import relevant packages

import pandas as pd
from scipy import stats

You are also provided with a dataset with national Air Quality Index (AQI) measurements by state over time for this analysis. `Pandas` was used to import the file `c4_epa_air_quality.csv` as a dataframe named `aqi`. As shown in this cell, the dataset has been automatically loaded in for you. You do not need to download the .csv file, or provide more code, in order to access the dataset and proceed with this lab. Please continue with this activity by completing the following instructions.

**Note:** For purposes of your analysis, you can assume this data is randomly sampled from a larger population.

#### Load Dataset

In [5]:
# IMPORT YOUR DATA
epa_data = pd.read_csv('c4_epa_air_quality.csv', index_col = 0)
epa_data = epa_data.dropna()

## Step 2: Data Exploration

### Before proceeding to your deliverables, explore your datasets.

Use the following space to surface descriptive statistics about your data. In particular, explore whether you believe the research questions you were given are readily answerable with this data.

In [6]:
# Use head() to show a sample of data

epa_data.head(10)

,date_local,state_name,county_name,city_name,local_site_name,parameter_name,units_of_measure,arithmetic_mean,aqi
0,2018-01-01,Arizona,Maricopa,Buckeye,BUCKEYE,Carbon monoxide,Parts per million,0.473684,7
1,2018-01-01,Ohio,Belmont,Shadyside,Shadyside,Carbon monoxide,Parts per million,0.263158,5
2,2018-01-01,Wyoming,Teton,Not in a city,Yellowstone National Park - Old Faithful Snow ...,Carbon monoxide,Parts per million,0.111111,2
3,2018-01-01,Pennsylvania,Philadelphia,Philadelphia,North East Waste (NEW),Carbon monoxide,Parts per million,0.300000,3
4,2018-01-01,Iowa,Polk,Des Moines,CARPENTER,Carbon monoxide,Parts per million,0.215789,3
5,2018-01-01,Hawaii,Honolulu,Not in a city,Kapolei,Carbon monoxide,Parts per million,0.994737,14
6,2018-01-01,Hawaii,Honolulu,Not in a city,Kapolei,Carbon monoxide,Parts per million,0.200000,2
8,2018-01-01,Hawaii,Honolulu,Honolulu,Honolulu,Carbon monoxide,Parts per million,0.400000,5
9,2018-01-01,Colorado,Larimer,Fort Collins,Fort Collins - CSU - S. Mason,Carbon monoxide,Parts per million,0.300000,6
10,2018-01-01,Minnesota,Dakota,Rosemount,Flint Hills Refinery 420,Carbon monoxide,Parts per million,0.247368,3


In [12]:
# check varibles
print(epa_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 257 entries, 0 to 259
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date_local        257 non-null    object 
 1   state_name        257 non-null    object 
 2   county_name       257 non-null    object 
 3   city_name         257 non-null    object 
 4   local_site_name   257 non-null    object 
 5   parameter_name    257 non-null    object 
 6   units_of_measure  257 non-null    object 
 7   arithmetic_mean   257 non-null    float64
 8   aqi               257 non-null    int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 20.1+ KB
None


In [13]:
# Use describe() to summarize AQI
epa_data['aqi'].describe()

count    257.000000
mean       6.782101
std        7.091422
min        0.000000
25%        2.000000
50%        5.000000
75%        9.000000
max       50.000000
Name: aqi, dtype: float64

In [20]:
# For a more thorough examination of observations by state use values_counts()

print(epa_data['state_name'].value_counts())

state_name
California              66
Arizona                 14
Ohio                    12
Florida                 12
Texas                   10
New York                10
Pennsylvania             9
Colorado                 9
Michigan                 9
Minnesota                7
New Jersey               6
Indiana                  5
Massachusetts            4
Oklahoma                 4
North Carolina           4
Nevada                   4
Maryland                 4
Connecticut              4
Virginia                 4
Utah                     3
Vermont                  3
Illinois                 3
Missouri                 3
Hawaii                   3
Iowa                     3
Wyoming                  3
Kentucky                 3
Alaska                   2
Rhode Island             2
Georgia                  2
Tennessee                2
Washington               2
Montana                  2
Maine                    2
Idaho                    2
New Mexico               2
District Of Colum

#### **Question 1: From the preceding data exploration, what do you recognize?**

A: I recognize that this data allows us to compare AQI across states, making it useful for hypothesis testing. However, the number of observations isn’t uniform across states, which could affect the reliability of certain comparisons.

## Step 3. Statistical Tests

Before you proceed, recall the following steps for conducting hypothesis testing:

1. Formulate the null hypothesis and the alternative hypothesis.<br>
2. Set the significance level.<br>
3. Determine the appropriate test procedure.<br>
4. Compute the p-value.<br>
5. Draw your conclusion.

### Hypothesis 1: ROA is considering a metropolitan-focused approach. Within California, they want to know if the mean AQI in Los Angeles County is statistically different from the rest of California.

Before proceeding with your analysis, it will be helpful to subset the data for your comparison.

In [22]:
# Create dataframes for each sample being compared in your test

california = epa_data[epa_data['state_name'] == "California"]
los_angeles = epa_data[epa_data['county_name'] == "Los Angeles"]


In [25]:
# Check head

los_angeles.head()

,date_local,state_name,county_name,city_name,local_site_name,parameter_name,units_of_measure,arithmetic_mean,aqi
33,2018-01-01,California,Los Angeles,Lancaster,Lancaster-Division Street,Carbon monoxide,Parts per million,0.394737,7
42,2018-01-01,California,Los Angeles,Santa Clarita,Santa Clarita,Carbon monoxide,Parts per million,0.394737,7
61,2018-01-01,California,Los Angeles,Pasadena,Pasadena,Carbon monoxide,Parts per million,0.789474,16
76,2018-01-01,California,Los Angeles,Los Angeles,LAX Hastings,Carbon monoxide,Parts per million,0.863158,17
109,2018-01-01,California,Los Angeles,Los Angeles,Los Angeles-North Main Street,Carbon monoxide,Parts per million,0.994737,17


#### Formulate your hypothesis:

**Formulate your null and alternative hypotheses:**

*   $H_0$: There is no difference in the mean AQI between Los Angeles County and the rest of California.
*   $H_A$: There is a difference in the mean AQI between Los Angeles County and the rest of California.


#### Set the significance level:

In [14]:
# For this analysis, the significance level is 5%

sig_val = 0.05

#### Determine the appropriate test procedure:

Here, you are comparing the sample means between two independent samples. Therefore, you will utilize a **two-sample  𝑡-test**.

#### Compute the P-value

In [35]:
# Compute your p-value here
t_stat, p_value = stats.ttest_ind(california['aqi'], los_angeles['aqi'], equal_var=False)
print(p_value)

0.11435587227090667


#### **Question 2. What is your P-value for hypothesis 1, and what does this indicate for your null hypothesis?**

In [36]:
# Extracting pvalue and make the test
if p_value < 0.05:
    
    print('p_value < 0.05, Reject Ho.')          # Ha: There is a difference in the mean
else:
    print('p_value > 0.05, Fail to reject Ho.')  # Ho: There is no difference in the mean

p_value > 0.05, Fail to reject Ho.


A: We fail to reject the null hypothesis because the p-value is greater than 5%. This means that there is no statistically significant difference in means

### Hypothesis 2: With limited resources, ROA has to choose between New York and Ohio for their next regional office. Does New York have a lower AQI than Ohio?

Before proceeding with your analysis, it will be helpful to subset the data for your comparison.

In [40]:
# Create dataframes for each sample being compared in your test
new_york = epa_data[epa_data['state_name'] == "New York"]
ohio = epa_data[epa_data['state_name'] == "Ohio"]

In [41]:
# Check head
new_york.head()
ohio.head()

,date_local,state_name,county_name,city_name,local_site_name,parameter_name,units_of_measure,arithmetic_mean,aqi
1,2018-01-01,Ohio,Belmont,Shadyside,Shadyside,Carbon monoxide,Parts per million,0.263158,5
12,2018-01-01,Ohio,Hamilton,Cincinnati,Taft NCore,Carbon monoxide,Parts per million,0.252632,3
22,2018-01-01,Ohio,Stark,Canton,Canton,Carbon monoxide,Parts per million,0.394737,6
51,2018-01-01,Ohio,Summit,Akron,NIHF STEM MS,Carbon monoxide,Parts per million,0.083333,3
59,2018-01-01,Ohio,Cuyahoga,Cleveland,GT Craig NCore,Carbon monoxide,Parts per million,0.250000,3


In [55]:
print(new_york['aqi'].mean())
print(ohio['aqi'].mean())

2.5
3.3333333333333335


**Formulate your null and alternative hypotheses:**

*   $H_0$: The mean AQI of New York is greater than or equal to that of Ohio.
*   $H_A$: The mean AQI of New York is **below** that of Ohio.


#### Significance Level (remains at 5%)

#### Determine the appropriate test procedure:

Here, you are comparing the sample means between two independent samples in one direction. Therefore, you will utilize a **two-sample  𝑡-test**.

#### Compute the P-value

In [43]:
# Compute your p-value here
t_stat, p_value = stats.ttest_ind(new_york['aqi'], ohio['aqi'], equal_var=False)
print(p_value)

0.06089300538386936


#### **Question 3. What is your P-value for hypothesis 2, and what does this indicate for your null hypothesis?**

In [44]:
# Your code here.
if p_value < 0.05:
    
    print('p_value < 0.05, Reject Ho.')          # Ha: There is a difference in the mean
else:
    print('p_value > 0.05, Fail to reject Ho.')  # Ho: There is no difference in the mean

p_value > 0.05, Fail to reject Ho.


A: We fail to reject the null hypothesis because the p-value is greater than 5%. This means that there is no statistically significant difference in means

###  Hypothesis 3: A new policy will affect those states with a mean AQI of 10 or greater. Will Michigan be affected by this new policy?

Before proceeding with your analysis, it will be helpful to subset the data for your comparison.

In [60]:
# Create dataframes for each sample being compared in your test
michigan = epa_data[epa_data['state_name'] == "Michigan"]
michigan['aqi'].mean()

np.float64(8.11111111111111)

**Formulate your null and alternative hypotheses here:**

*   $H_0$: The mean AQI of Michigan is less than or equal to 10.
*   $H_A$: The mean AQI of Michigan is greater than 10.


#### Significance Level (remains at 5%)

#### Determine the appropriate test procedure:

Here, you are comparing one sample mean relative to a particular value in one direction. Therefore, you will utilize a **one-sample  𝑡-test**. 

#### Compute the P-value

In [62]:
# Compute your p-value here
t_stat, p_value = stats.ttest_ind(michigan['aqi'], 10, equal_var=False)

if p_value < 0.05:
    
    print('p_value < 0.05, Reject Ho.')          # Ha: There is a difference in the mean
else:
    print('p_value > 0.05, Fail to reject Ho.')  # Ho: There is no difference in the mean

p_value > 0.05, Fail to reject Ho.


#### **Question 4. What is your P-value for hypothesis 3, and what does this indicate for your null hypothesis?**

A: A: We fail to reject the null hypothesis because the p-value is greater than 5%. This means that there is no statistically significant difference between 10 and Michigan's mean aqi

## Step 4. Results and Evaluation

Now that you've completed your statistical tests, you can consider your hypotheses and the results you gathered.

#### **Question 5. Did your results show that the AQI in Los Angeles County was statistically different from the rest of California?**

A: no

#### **Question 6. Did New York or Ohio have a lower AQI?**

A: New york

#### **Question 7: Will Michigan be affected by the new policy impacting states with a mean AQI of 10 or greater?**



A: They could be, as their mean aqi is below 10 but is not statistically different from 10.

# Conclusion

**What are key takeaways from this project?**

A: how to do hypthesis testing using scipy

**What would you consider presenting to your manager as part of your findings?**

A: that there is no statistically significant difference between Los Angeles and California, New York has a lower AQI than Ohio, making it a better location for an office (even if it's not statistically different from Ohio), and that Michigan could be impacted by new regulations.


**What would you convey to external readers?**

A: This repot shows the power of data-driven decisions. By using statistical analysis, we can allocate resources to effectively to maximize public health benefits.
